NYC Uber/Lyft Rides
===================

The NYC Taxi dataset is a timeless classic.  

Interestingly there is a new variant.  The NYC Taxi and Livery Commission requires data from all ride-share services in the city of New York.  This includes private limosine services, van services, and a new category "High Volume For Hire Vehicle" services, those that dispatch 10,000 rides per day or more.  This is a special category defined for Uber and Lyft.  

This data is available here:

In [ ]:
import coiled

cluster = coiled.Cluster(
    n_workers=30,
    backend_options={"region_name": "us-east-2"},
    account="events",
    name="uber-lyft",
    shutdown_on_close=False,
)

client = cluster.get_client()

In [ ]:
client.restart()

In [ ]:
import dask
import pandas
import dask.dataframe as dd

dask.config.set({"dataframe.convert-string": True})  # use PyArrow strings by default

df = dd.read_parquet(
    "s3://coiled-datasets/uber-lyft-tlc/",
    storage_options={"anon": True},
)
df.head()

Play time
---------

We actually don't know what to expect from this dataset.  No one in our team has spent much time inspecting it.  We'd like to solicit help from you, new Dask user, to uncover some interesting insights.  

Care to explore and report your findings?

In [ ]:
df = df.persist()

df.columns

In [ ]:
df.base_passenger_fare.sum().compute() / 1e9

## Basic statistics

In [ ]:
total = df[["base_passenger_fare", "driver_pay", "tips", "trip_miles"]].sum()
average = df[["base_passenger_fare", "driver_pay", "tips", "trip_miles"]].mean()



In [ ]:
total

In [ ]:
average

In [ ]:
total, average = dask.compute(total, average)

## Tipping Practices

In [ ]:
df[df.tips != 0].tips.mean().compute()

In [ ]:
(df.tips != 0).mean().compute()

## Broken down by carrier

In [ ]:
df.hvfhs_license_num.value_counts().compute()

In [ ]:
df["tipped"] = df.tips != 0

df.groupby("hvfhs_license_num").tipped.mean().compute()

In [ ]:
client.restart()